## Human in the Loop

With the help of human in the loop we as a user can be a part of the team and can provide our feedback to the team. This can be achieved by two ways:

1. During the team run: using UserProxyAgent
2. Once the team terminate: Feedback for the next iteration.

In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
load_dotenv()

model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')

### Human-in-the-loop during the Run

We use `UserProxyAgent` which is a built-in agent that act as a proxy for the user to provide the feedback to the team.

The diagram below explains how we can use `UserProxyAgent` in the team flow, to get the feedback from the user.

![alt text](human-in-the-loop-user-proxy.svg)

In [2]:
assistant = AssistantAgent(
    name = 'Asssitant',
    model_client=model_client,
    system_message='You are a helpful assistant'
)

user_proxy = UserProxyAgent(
    name='user',
    description='A proxy agent that represent the user',
    input_func=input
)

## termination condition
text_mention_termination = TextMentionTermination('APPROVE')

## team
team = RoundRobinGroupChat(
    participants=[assistant, user_proxy],
    termination_condition=text_mention_termination
)

In [3]:
from autogen_agentchat.ui import Console

await Console(team.run_stream(task='Write a 4 line poem about life'))

---------- TextMessage (user) ----------
Write a 4 line poem about life
---------- TextMessage (Asssitant) ----------
Life's a dance of joy and strife,  
Whispers of dreams, the pulse of life.  
In every tear and every smile,  
We weave our stories, mile by mile.
---------- TextMessage (user) ----------
APPROVE


TaskResult(messages=[TextMessage(id='424ba079-2f9e-4f42-b09a-10ce619d7fe5', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 5, 53, 3, 582371, tzinfo=datetime.timezone.utc), content='Write a 4 line poem about life', type='TextMessage'), TextMessage(id='0d9c5a7a-c51a-4fc0-885b-33ef91ccc377', source='Asssitant', models_usage=RequestUsage(prompt_tokens=25, completion_tokens=39), metadata={}, created_at=datetime.datetime(2025, 8, 8, 5, 53, 6, 227049, tzinfo=datetime.timezone.utc), content="Life's a dance of joy and strife,  \nWhispers of dreams, the pulse of life.  \nIn every tear and every smile,  \nWe weave our stories, mile by mile.", type='TextMessage'), UserInputRequestedEvent(id='9dd7001d-04b7-4a1c-a95e-38cbb48ab876', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 5, 53, 6, 228553, tzinfo=datetime.timezone.utc), request_id='53021adc-9e96-47fe-bbe3-930937f0babc', content='', type='UserInputRequestedEvent')

In [4]:
await Console(team.run_stream(task='Write a 4 line poem about India'))

---------- TextMessage (user) ----------
Write a 4 line poem about India
---------- TextMessage (Asssitant) ----------
From rivers' flow to mountains high,  
Colors burst beneath the sky.  
Cultures blend in harmony,  
India's heart, a tapestry.
---------- TextMessage (user) ----------
mentione more about india heritage
---------- TextMessage (Asssitant) ----------
In ancient scripts and temples grand,  
Where wisdom flourished from the land.  
Festivals bright, traditions deep,  
India's heritage, a treasure to keep.  

With dance and song, the past embraced,  
In every corner, history traced.  
From Vedas’ lore to art's fine thread,  
A vibrant legacy, where dreams are bred.
---------- TextMessage (user) ----------

---------- TextMessage (Asssitant) ----------
It seems you may not have included a request or a question in your last message. How can I assist you further? If you'd like another poem or anything else, feel free to let me know!
---------- TextMessage (user) ----------
APP

TaskResult(messages=[TextMessage(id='a0a25931-bb11-4a30-a321-9da8bf870371', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 5, 53, 43, 405835, tzinfo=datetime.timezone.utc), content='Write a 4 line poem about India', type='TextMessage'), TextMessage(id='e304802a-1341-4944-a7d1-2954ae8b73dd', source='Asssitant', models_usage=RequestUsage(prompt_tokens=89, completion_tokens=30), metadata={}, created_at=datetime.datetime(2025, 8, 8, 5, 53, 44, 728129, tzinfo=datetime.timezone.utc), content="From rivers' flow to mountains high,  \nColors burst beneath the sky.  \nCultures blend in harmony,  \nIndia's heart, a tapestry.", type='TextMessage'), UserInputRequestedEvent(id='0fa12baa-c09d-42e2-a7a7-89ec446f8043', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 5, 53, 44, 730274, tzinfo=datetime.timezone.utc), request_id='145a4bc9-5478-44ff-b8df-13602151cdaf', content='', type='UserInputRequestedEvent'), TextMessage(i

#### The problem with this way is that the state of the team is unstable as it blocks the progress until the user provides a feedback or approve the result.

#### Because of this blocking approach, it is recommended that we use it for short interactions.


### Human-in-the-loop in the Next Run

This approach is used in an interactive loop, where the team runs until termination and this approach is useful for persistant session with async communication between the team and the user. Everytime an execution completes, state of the team is updated and resumes from where it was left.

![alt text](human-in-the-loop-termination.svg)

We can use this with either:
1. Maximum number of turns so that team always terminate after certain number of runs or
2. Use termination conditions such as TextMentionTermination and HandoffTermination to allow the team to decide when to stop and give the control back to the user.